In [88]:
# Import libraries
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import arrays_zip, array_distinct, col, explode, substring, lit, when, size, from_unixtime, unix_timestamp, expr, sum, avg, max, min, mean, count, countDistinct, datediff

In [2]:
# spark session & spark context
sc = SparkContext()
spark = SparkSession.builder.appName('iovio').getOrCreate()

In [3]:
df_users = spark.read.csv('/Users/esau/Downloads/IOVIO/users.csv', \
                            header = True)

In [13]:
df_result = spark.read.json('/Users/esau/Downloads/IOVIO/results/*-result.json')

In [14]:
df_result.printSchema()
df_result.show(10)
df_result.count()

root
 |-- attachments: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- source: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- description: string (nullable = true)
 |-- historyId: string (nullable = true)
 |-- labels: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- value: string (nullable = true)
 |-- links: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |    |    |-- url: string (nullable = true)
 |-- name: string (nullable = true)
 |-- parameters: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- value: string (nullable = true)
 |-- stage: string (nullable = true)
 |-- start: long (nullable 

1722

In [15]:
# Count and Add the number of steps per test
df_result = df_result.withColumn('steps_count', size(array_distinct('steps')))

In [79]:
# Explode into rows by attachment source   
df_result2 = df_result.select(df_result.historyId, df_result.uuid, df_result.start, df_result.stop, df_result.description, df_result.steps_count, explode(df_result.attachments))

In [80]:
# Get attachment type and name
df_result2 = df_result2.select(df_result2.historyId, df_result2.uuid, df_result2.start, df_result2.stop, df_result2.description, df_result2.steps_count,\
                               df_result2['col.name'].alias('attach_name'), df_result2['col.source'].alias('attach_source'))

In [81]:
# Unix timestamp to datetime 
df_result2 = df_result2.withColumn('start_date', from_unixtime((col('start')/1000)))
df_result2 = df_result2.withColumn('stop_date', from_unixtime((col('stop')/1000)))
df_result2 = df_result2.withColumn('time_dif', expr("(stop - start)/1000"))
columns_to_drop = ['start', 'stop']
df_result2 = df_result2.drop(*columns_to_drop)
df_result2.show()

+--------------------+--------------------+-----------+-----------+-----------+--------------------+-------------------+-------------------+--------+
|           historyId|                uuid|description|steps_count|attach_name|       attach_source|         start_date|          stop_date|time_dif|
+--------------------+--------------------+-----------+-----------+-----------+--------------------+-------------------+-------------------+--------+
|189e9f7a8cb436a35...|fa2259dc-8db5-47a...|        SEO|         71|   wdio-log|ddcaee06-2d6c-421...|2021-01-04 18:39:35|2021-01-04 18:40:03|  28.129|
|189e9f7a8cb436a35...|fa2259dc-8db5-47a...|        SEO|         71|   test-log|33fa3e3a-3948-417...|2021-01-04 18:39:35|2021-01-04 18:40:03|  28.129|
|4835592a049391492...|f4af646f-821c-4ea...|        SEO|         71|   wdio-log|5478b2f2-3a69-474...|2021-01-04 18:39:35|2021-01-04 18:40:02|  26.564|
|4835592a049391492...|f4af646f-821c-4ea...|        SEO|         71|   test-log|f235d8a8-51d6-435...|

In [43]:
df_attachments = sc.wholeTextFiles('/Users/esau/Downloads/IOVIO/results/*.txt')
df_attachments = df_attachments.toDF()
df_attachments = df_attachments.withColumnRenamed('_1', 'file')\
                                .withColumnRenamed('_2', 'data')

In [44]:
df_attachments.printSchema()
df_attachments.show()
df_attachments.count()

root
 |-- file: string (nullable = true)
 |-- data: string (nullable = true)

+--------------------+--------------------+
|                file|                data|
+--------------------+--------------------+
|file:/Users/esau/...|2021-01-05T00:45:...|
|file:/Users/esau/...|2021-01-05T00:19:...|
|file:/Users/esau/...|2021-01-05T00:13:...|
|file:/Users/esau/...|2021-01-05T00:20:...|
|file:/Users/esau/...|2021-01-04T23:53:...|
|file:/Users/esau/...|{
  "paymentMetho...|
|file:/Users/esau/...|2021-01-05T00:12:...|
|file:/Users/esau/...|2021-01-05T01:15:...|
|file:/Users/esau/...|2021-01-05T00:16:...|
|file:/Users/esau/...|2021-01-05T00:12:...|
|file:/Users/esau/...|2021-01-05T00:07:...|
|file:/Users/esau/...|2021-01-04T23:51:...|
|file:/Users/esau/...|2021-01-05T00:24:...|
|file:/Users/esau/...|2021-01-05T00:19:...|
|file:/Users/esau/...|2021-01-05T00:55:...|
|file:/Users/esau/...|{
  "checkoutStep...|
|file:/Users/esau/...|2021-01-05T00:16:...|
|file:/Users/esau/...|2021-01-05T00:20:...

3537

In [45]:
# Getting filename
df_attachments = df_attachments.select('file',\
                                substring('File', 42, 51).alias('attach_source'),\
                                'data')
df_attachments.show()

+--------------------+--------------------+--------------------+
|                file|       attach_source|                data|
+--------------------+--------------------+--------------------+
|file:/Users/esau/...|000cc5a5-c127-43d...|2021-01-05T00:45:...|
|file:/Users/esau/...|001f9167-d504-427...|2021-01-05T00:19:...|
|file:/Users/esau/...|0023d6c8-7821-452...|2021-01-05T00:13:...|
|file:/Users/esau/...|00245c47-ca80-4fa...|2021-01-05T00:20:...|
|file:/Users/esau/...|00504b4f-3022-455...|2021-01-04T23:53:...|
|file:/Users/esau/...|00611107-5e87-48f...|{
  "paymentMetho...|
|file:/Users/esau/...|0065be4c-b397-4bc...|2021-01-05T00:12:...|
|file:/Users/esau/...|0085b452-521e-431...|2021-01-05T01:15:...|
|file:/Users/esau/...|00a2791a-6b36-421...|2021-01-05T00:16:...|
|file:/Users/esau/...|00c0470e-2a0c-410...|2021-01-05T00:12:...|
|file:/Users/esau/...|00c96c7e-ee06-45c...|2021-01-05T00:07:...|
|file:/Users/esau/...|00dd1974-44bc-4bb...|2021-01-04T23:51:...|
|file:/Users/esau/...|00f

In [46]:
# Creating new columns with the string to match
df_attachments = df_attachments.withColumn('str1', lit(' STORE_ID'))
df_attachments = df_attachments.withColumn('str2', lit('storeId'))
df_attachments.show()

+--------------------+--------------------+--------------------+---------+-------+
|                file|       attach_source|                data|     str1|   str2|
+--------------------+--------------------+--------------------+---------+-------+
|file:/Users/esau/...|000cc5a5-c127-43d...|2021-01-05T00:45:...| STORE_ID|storeId|
|file:/Users/esau/...|001f9167-d504-427...|2021-01-05T00:19:...| STORE_ID|storeId|
|file:/Users/esau/...|0023d6c8-7821-452...|2021-01-05T00:13:...| STORE_ID|storeId|
|file:/Users/esau/...|00245c47-ca80-4fa...|2021-01-05T00:20:...| STORE_ID|storeId|
|file:/Users/esau/...|00504b4f-3022-455...|2021-01-04T23:53:...| STORE_ID|storeId|
|file:/Users/esau/...|00611107-5e87-48f...|{
  "paymentMetho...| STORE_ID|storeId|
|file:/Users/esau/...|0065be4c-b397-4bc...|2021-01-05T00:12:...| STORE_ID|storeId|
|file:/Users/esau/...|0085b452-521e-431...|2021-01-05T01:15:...| STORE_ID|storeId|
|file:/Users/esau/...|00a2791a-6b36-421...|2021-01-05T00:16:...| STORE_ID|storeId|
|fil

In [47]:
# Gettin start index of the string to match
df_stores_ids = df_attachments.withColumn('store_id_idx', expr('locate(str1, data) - 1'))
df_stores_ids = df_stores_ids.withColumn('store_id_idx2', expr('locate(str2, data) - 1'))
df_stores_ids.show()

+--------------------+--------------------+--------------------+---------+-------+------------+-------------+
|                file|       attach_source|                data|     str1|   str2|store_id_idx|store_id_idx2|
+--------------------+--------------------+--------------------+---------+-------+------------+-------------+
|file:/Users/esau/...|000cc5a5-c127-43d...|2021-01-05T00:45:...| STORE_ID|storeId|          -1|          705|
|file:/Users/esau/...|001f9167-d504-427...|2021-01-05T00:19:...| STORE_ID|storeId|          -1|           -1|
|file:/Users/esau/...|0023d6c8-7821-452...|2021-01-05T00:13:...| STORE_ID|storeId|          -1|           -1|
|file:/Users/esau/...|00245c47-ca80-4fa...|2021-01-05T00:20:...| STORE_ID|storeId|          -1|        40428|
|file:/Users/esau/...|00504b4f-3022-455...|2021-01-04T23:53:...| STORE_ID|storeId|         649|           -1|
|file:/Users/esau/...|00611107-5e87-48f...|{
  "paymentMetho...| STORE_ID|storeId|          -1|           -1|
|file:/Use

In [48]:
# Filter rows where string not found
df_stores_ids2 = df_stores_ids.filter(((col('store_id_idx') != -1) | (col('store_id_idx2') != -1)))

In [49]:
# Integrating start index in a single column
df_stores_ids2 = df_stores_ids2.withColumn('store_id', when(df_stores_ids2.store_id_idx > -1,\
                                                            df_stores_ids2['data'].substr(col('store_id_idx') + 12, lit(5))).\
                                                            otherwise(df_stores_ids2['data'].substr(col('store_id_idx2') + 9, lit(5))))

In [50]:
# Delete columns
columns_to_drop = ['file', 'data', 'str1', 'str2', 'store_id_idx', 'store_id_idx2']
df_stores_ids2 = df_stores_ids2.drop(*columns_to_drop)

In [51]:
df_stores_ids2.show(20, False)
df_stores_ids2.count()

+---------------------------------------------------+--------+
|attach_source                                      |store_id|
+---------------------------------------------------+--------+
|000cc5a5-c127-43d4-af79-f4d4418447ff-attachment.txt|20006   |
|00245c47-ca80-4fa9-a55d-8aea8f42a788-attachment.txt|20017   |
|00504b4f-3022-4555-866a-33aa1ad7a5ff-attachment.txt|20017   |
|0065be4c-b397-4bc5-bda5-c029f836db02-attachment.txt|20027   |
|0085b452-521e-431f-851a-81e7edbb680c-attachment.txt|20006   |
|00c0470e-2a0c-4104-817f-8dd371cbf459-attachment.txt|20006   |
|00c96c7e-ee06-45cb-a552-f6a7b5f5fb65-attachment.txt|20006   |
|00dd1974-44bc-4bb6-91fb-a74d64b4d7c6-attachment.txt|20006   |
|010dbfbf-4648-4c8a-8287-1dadfddcc26b-attachment.txt|20006   |
|012c09cc-b88a-45f6-8815-c25be6167ef8-attachment.txt|20006   |
|0140719c-5475-4a6b-bcb1-9e799fc46faa-attachment.txt|20006   |
|01409134-db6c-45fe-b631-0b493243dfcd-attachment.txt|20006   |
|01499fb0-0908-49db-a4d2-854fc9326167-attachment.txt|20

2477

In [82]:
# Considering only the attachments type 'test-log'
df_analytics = df_result2.filter(df_result2.attach_name == 'test-log')
df_analytics.show()
df_analytics.count()

+--------------------+--------------------+--------------------+-----------+-----------+--------------------+-------------------+-------------------+--------+
|           historyId|                uuid|         description|steps_count|attach_name|       attach_source|         start_date|          stop_date|time_dif|
+--------------------+--------------------+--------------------+-----------+-----------+--------------------+-------------------+-------------------+--------+
|189e9f7a8cb436a35...|fa2259dc-8db5-47a...|                 SEO|         71|   test-log|33fa3e3a-3948-417...|2021-01-04 18:39:35|2021-01-04 18:40:03|  28.129|
|4835592a049391492...|f4af646f-821c-4ea...|                 SEO|         71|   test-log|f235d8a8-51d6-435...|2021-01-04 18:39:35|2021-01-04 18:40:02|  26.564|
|e0e3d7850b3e748b3...|e83ab84e-b94d-473...|                 SEO|         71|   test-log|2950f201-b47d-4de...|2021-01-04 18:39:35|2021-01-04 18:40:04|  29.278|
|65a0ce8472f7a5135...|825bc5e2-2b16-49d...|   

1722

In [83]:
# Join Results with Logs to get STORE_ID
df_analytics2 = df_analytics.join(df_stores_ids2, ['attach_source'], 'inner')
df_analytics2.show()
df_analytics2.count()

+--------------------+--------------------+--------------------+--------------------+-----------+-----------+-------------------+-------------------+--------+--------+
|       attach_source|           historyId|                uuid|         description|steps_count|attach_name|         start_date|          stop_date|time_dif|store_id|
+--------------------+--------------------+--------------------+--------------------+-----------+-----------+-------------------+-------------------+--------+--------+
|00504b4f-3022-455...|53e2cf853561dc0a7...|6eb6b21a-1263-419...|Payment Page - Bi...|         13|   test-log|2021-01-04 17:53:10|2021-01-04 17:53:41|    30.4|   20017|
|0065be4c-b397-4bc...|00303848cd5f360ed...|0398ae5b-de12-4e8...|Customer service ...|          4|   test-log|2021-01-04 18:12:44|2021-01-04 18:12:56|  11.882|   20027|
|0085b452-521e-431...|9987efb6f9b53c568...|cebc7443-8246-447...|   Country Selection|         11|   test-log|2021-01-04 19:15:57|2021-01-04 19:16:12|  15.155|  

1721

In [87]:
df_analytics2.groupBy('historyId')

1721

In [89]:
# 1. How many test cases are executed per locale?

df_analytics2.groupBy('store_id').agg(countDistinct('historyId').alias('count_test_cases')).orderBy(col('count_test_cases').desc()).show(50, truncate=False)

'''
+--------+----------------+
|store_id|count_test_cases|
+--------+----------------+
|20006   |673             |
|20002   |289             |
|20017   |119             |
|20027   |94              |
|20023   |58              |
|20019   |41              |
|20004   |38              |
|20014   |25              |
|20020   |24              |
|20015   |20              |
|20005   |17              |
|20021   |11              |
|20011   |11              |
|20009   |10              |
|20018   |8               |
|20007   |8               |
|20001   |8               |
|20016   |7               |
|20010   |6               |
|20022   |6               |
|20026   |6               |
|20003   |6               |
|20024   |6               |
|20025   |6               |
|20008   |6               |
|20012   |6               |
|20013   |6               |
+--------+----------------+
'''

+--------+----------------+
|store_id|count_test_cases|
+--------+----------------+
|20006   |673             |
|20002   |289             |
|20017   |119             |
|20027   |94              |
|20023   |58              |
|20019   |41              |
|20004   |38              |
|20014   |25              |
|20020   |24              |
|20015   |20              |
|20005   |17              |
|20021   |11              |
|20011   |11              |
|20009   |10              |
|20018   |8               |
|20007   |8               |
|20001   |8               |
|20016   |7               |
|20010   |6               |
|20022   |6               |
|20026   |6               |
|20003   |6               |
|20024   |6               |
|20025   |6               |
|20008   |6               |
|20012   |6               |
|20013   |6               |
+--------+----------------+



In [85]:
# Join Results & Logs With Users to get manager name

df_analytics3 = df_analytics2.join(df_users, ['store_id'], 'inner')
df_analytics3.show()
df_analytics3.count()

+--------+--------------------+--------------------+--------------------+--------------------+-----------+-----------+-------------------+-------------------+--------+------------------+
|store_id|       attach_source|           historyId|                uuid|         description|steps_count|attach_name|         start_date|          stop_date|time_dif|      test_manager|
+--------+--------------------+--------------------+--------------------+--------------------+-----------+-----------+-------------------+-------------------+--------+------------------+
|   20017|00504b4f-3022-455...|53e2cf853561dc0a7...|6eb6b21a-1263-419...|Payment Page - Bi...|         13|   test-log|2021-01-04 17:53:10|2021-01-04 17:53:41|    30.4|Gabriella Thompson|
|   20027|0065be4c-b397-4bc...|00303848cd5f360ed...|0398ae5b-de12-4e8...|Customer service ...|          4|   test-log|2021-01-04 18:12:44|2021-01-04 18:12:56|  11.882|        Jose Lopez|
|   20006|0085b452-521e-431...|9987efb6f9b53c568...|cebc7443-8246

1721

In [91]:
# 2. What is the name of the test manager who's test cases sum the highest number of steps, and how many steps put him/her in the top 1 position?

df_analytics3.groupBy('test_manager').agg(sum('steps_count').alias('sum_total_steps')).orderBy(col('sum_total_steps').desc()).show(1, False)

'''
+-------------+---------------+
|test_manager |sum_total_steps|
+-------------+---------------+
|Sharon Martin|9096           |
+-------------+---------------+
'''

+-------------+---------------+
|test_manager |sum_total_steps|
+-------------+---------------+
|Sharon Martin|9096           |
+-------------+---------------+
only showing top 1 row



In [97]:
# 3. What is the average duration in seconds across all test cases?

df_analytics3.groupBy().avg('time_dif').alias('test_avg_duration').show() 

'''
+-----------------+
|    avg(time_dif)|
+-----------------+
|29.80986635676936|
+-----------------+
'''

+-----------------+
|    avg(time_dif)|
+-----------------+
|29.80986635676936|
+-----------------+



In [103]:
# 4. Which features contains the largest number across all test cases?

df_analytics3.groupBy('description').agg(countDistinct('historyId').alias('count_of_tests')).orderBy(col('count_of_tests').desc()).show(5, False)

'''
+--------------------+--------------+
|         description|count_of_tests|
+--------------------+--------------+
|        Shopping Bag|           136|
'''

+-----------------------------+--------------+
|description                  |count_of_tests|
+-----------------------------+--------------+
|Shopping Bag                 |136           |
|Filters on PLP and search PLP|95            |
|SEO                          |91            |
|Datalayer - events           |81            |
|PLP                          |73            |
+-----------------------------+--------------+
only showing top 5 rows



'\n+--------------------+--------------+\n|         description|count_of_tests|\n+--------------------+--------------+\n|        Shopping Bag|           136|\n'